# Todo
* Sensitivity 250 500 1000
* For the 500, take top 10% heightest weight
* Take that threshold for other topics

https://miningthedetails.com/blog/python/lda/GensimLDA/
https://markroxor.github.io/gensim/static/notebooks/lda_training_tips.html

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#import tempfile
#TEMP_FOLDER = tempfile.gettempdir()
#print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('retina')

from sklearn import preprocessing

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.test.utils import datapath

In [2]:
# The model with model_name we want to load
def load_model(model_name):
    current_path = os.getcwd()
    current_path = Path(current_path)
    load_path = current_path.parent / "./Data/LDAModel/{}/{}".format(model_name,model_name)
    load_path = str(load_path)
    lda = gensim.models.ldamulticore.LdaMulticore.load(load_path)
    return lda

In [3]:
model_name = "matrix1_100topic"
matrix1_100topic = load_model(model_name)

In [4]:
model_name = "matrix1_250topic"
matrix1_250topic = load_model(model_name)

In [5]:
model_name = "matrix1_500topic"
matrix1_500topic = load_model(model_name)

In [6]:
model_name = "matrix1_750topic"
matrix1_750topic = load_model(model_name)

In [7]:
model_name = "matrix1_1000topic"
matrix1_1000topic = load_model(model_name)

In [8]:
model_name = "matrix1_1500topic"
matrix1_1500topic = load_model(model_name)

In [22]:
matrix1_750topic.print_topics()

[(603,
  '0.080*"HL4080028" + 0.036*"MCP3983624" + 0.031*"RCII.GH" + 0.011*"NSM4013441" + 0.011*"IO4117248" + 0.011*"WHR4099800" + 0.011*"NES3879264" + 0.009*"IGT.GI" + 0.009*"WLT4062096" + 0.009*"BTU3905482"'),
 (568,
  '0.003*"PBR4006643" + 0.003*"AAPL4001809" + 0.003*"VZ4050437" + 0.003*"PBR3688575" + 0.003*"TWC3674494" + 0.003*"CDFA.GC" + 0.002*"JPM3999853" + 0.002*"CCCX3917899" + 0.002*"GE.HEE" + 0.002*"AEE.HV"'),
 (22,
  '0.021*"NIEF4047034" + 0.004*"RIG.GX" + 0.003*"PVA.GD" + 0.003*"HL4080028" + 0.003*"PBR3688575" + 0.003*"AAPL4001809" + 0.003*"CLF.AC" + 0.003*"OKE.AA" + 0.002*"SWY.GN" + 0.002*"KO4066189"'),
 (96,
  '0.019*"PPL.IC" + 0.014*"RRD3674729" + 0.010*"CLF.AB" + 0.005*"PPL.IR" + 0.005*"RAI3922486" + 0.005*"GEF3677223" + 0.004*"DISH.GZ" + 0.003*"PFE3690876" + 0.003*"CCCX3917899" + 0.003*"VALE3701084"'),
 (134,
  '0.063*"HCP.JK" + 0.041*"JRCC.GG" + 0.013*"KRI.GD" + 0.009*"TEG4071296" + 0.008*"ALU3697111" + 0.007*"ETHN.GB" + 0.007*"RPIZ.AA" + 0.007*"ARCC4071336" + 0.006*"B

In [28]:
matrix1_750topic.show_topic(603)

[('HL4080028', 0.07980591),
 ('MCP3983624', 0.036443215),
 ('RCII.GH', 0.031155517),
 ('NSM4013441', 0.011121476),
 ('IO4117248', 0.011079177),
 ('WHR4099800', 0.011063738),
 ('NES3879264', 0.010838084),
 ('IGT.GI', 0.00911228),
 ('WLT4062096', 0.008906558),
 ('BTU3905482', 0.008750132)]

In [37]:
matrix1_750topic.get_topic_terms(50)

[(7999, 0.01583518),
 (9732, 0.012683756),
 (3552, 0.004808716),
 (10244, 0.0043968004),
 (3576, 0.0039317133),
 (229, 0.0038239034),
 (11488, 0.003514054),
 (9725, 0.0032417963),
 (10359, 0.0025387409),
 (261, 0.002467031)]

In [41]:
matrix1_750topic.get_term_topics(3552)

[]